In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
import re
import cv2

In [ ]:
import torch
import torchvision

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensor 

In [ ]:
from matplotlib.image import imread

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pwd

In [ ]:
ls

In [ ]:
cd ..

In [ ]:
cd input/global-wheat-detection

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
test = pd.read_csv('sample_submission.csv')

In [ ]:
train.head()

In [ ]:
def append_ext(fn):
    return fn+".jpg"
train["image_id"]=train["image_id"].apply(append_ext)


In [ ]:
train['bbox'].count()

In [ ]:
train['xmin'] = None
train['ymin'] = None
train['Width'] = None
train['Height'] = None

In [ ]:
def expand_bbox(x):
    r = np.array(re.findall("([0-9]+[.]?[0-9]*)", x))
    if len(r) == 0:
        r = [-1, -1, -1, -1]
    return r

train[['xmin', 'ymin', 'Width', 'Height']] = np.stack(train['bbox'].apply(lambda x: expand_bbox(x)))
train.drop(columns=['bbox'], inplace=True)
train['xmin'] = train['xmin'].astype(np.float)
train['ymin'] = train['ymin'].astype(np.float)
train['Width'] = train['Width'].astype(np.float)
train['Height'] = train['Height'].astype(np.float)

In [ ]:
train.head()

In [ ]:
train['source'].unique()

In [ ]:
sns.countplot(x = 'source', data=train)

# THIS SHOWS THE IMBALANCE IN DATASET

In [ ]:
sns.heatmap(train.isnull(), yticklabels=False, cbar=False, cmap='viridis')

# NO NULL VALUES

In [ ]:
train.head()

In [ ]:
train.head()

In [ ]:
train.drop('width', axis=1, inplace=True)

In [ ]:
train.drop('height', axis=1, inplace=True)

In [ ]:
pwd

In [ ]:
data_dir = '/kaggle/input/global-wheat-detection'

In [ ]:
os.listdir(data_dir)

In [ ]:
train_path = data_dir + '/train/'
test_path = data_dir + '/test/'

In [ ]:
a = os.listdir(test_path)[0]

In [ ]:
a

In [ ]:
sam_test = test_path + '348a992bb.jpg'

In [ ]:
sam_tensor = imread(sam_test)

In [ ]:
test_img = test_path+ '51b3e36ab.jpg'

In [ ]:
test_img = imread(test_img)

In [ ]:
test_img.shape

In [ ]:
plt.imshow(test_img)

In [ ]:
image_shape = (224,224,3)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(
    rescale=1./255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=90,
    width_shift_range=0.15,
    height_shift_range=0.15,
    horizontal_flip=True,
    zoom_range=[0.9, 1.25],
    brightness_range=[0.5, 1.5]
)

In [0]:
batch_size = 64

In [ ]:
test_image_generator = ImageDataGenerator(
    rescale=1./255)

In [ ]:
def append_ext(fn):
    return fn+".jpg"
test["image_id"]=test["image_id"].apply(append_ext)

train_gen = image_gen.flow_from_dataframe(
    dataframe=train,
    directory=train_path,
    x_col='image_id',
    y_col='source',
    target_size=image_shape[:2],
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    class_mode='categorical'
)

In [0]:
total_samples = 146704

In [ ]:
test_generator = test_image_generator.flow_from_dataframe(
    dataframe = test,
    directory=test_path,
     x_col='image_id',
    target_size=(1024,1024),
    shuffle=False,
    batch_size=8,
    class_mode=None
)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.callbacks import LearningRateScheduler
from keras.metrics import *
# v4

ACCURACY_LIST = []
from keras.applications.resnet50 import ResNet50
from keras.layers import GlobalMaxPooling2D
from keras.models import Model

In [ ]:
METRICS = [
     
      BinaryAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
]

def output_custom_model(prebuilt_model):
    print(f"Processing {prebuilt_model}")
    prebuilt = prebuilt_model(include_top=False,
                            input_shape=(1024,1024,3),
                            weights='imagenet')
    output = prebuilt.output
    output = GlobalMaxPooling2D()(output)
    output = Dense(128, activation='relu')(output)
    output = Dropout(0.4)(output)
    output = Dense(7, activation='softmax')(output)

    model = Model(inputs=prebuilt.input, outputs=output)
    model.compile(optimizer='adam', loss=categorical_crossentropy,
              metrics=METRICS)
    return model

In [ ]:
def scheduler(epoch):
    if epoch < 5:
        return 0.0001
    else:
        print(f"Learning rate reduced to {0.0001 * np.exp(0.5 * (5 - epoch))}")
        return 0.0001 * np.exp(0.5 * (5 - epoch))
    
custom_callback = LearningRateScheduler(scheduler)


In [ ]:
resnet_custom_model = output_custom_model(ResNet50)


resnet_custom_model = output_custom_model(ResNet50)
resnet_history = resnet_custom_model.fit_generator(train_gen,
                                 epochs=1,callbacks=[custom_callback]
                                 
                              )

  resnet_history.history

resnet_custom_model.save_weights('wheat_detection.h5')

In [ ]:
pwd

In [0]:
test = pd.read_csv('sample_submission.csv')

In [ ]:
test.head()

In [ ]:
cd ../weights

In [ ]:
resnet_custom_model.load_weights('wheat_detection.h5')

In [ ]:
sam_tensor.shape

In [ ]:
sam_tensor = np.expand_dims(sam_tensor,axis=0)

In [ ]:
preds = resnet_custom_model.predict(test_generator)

In [ ]:
preds.min()

In [ ]:
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.measure import label, regionprops
from PIL import Image, ImageDraw
from ast import literal_eval
from tqdm.notebook import tqdm


In [ ]:
IMG_WIDTH = 512
IMG_HEIGHT = 512

In [ ]:
SC_FACTOR = int(1024 / IMG_WIDTH)


In [ ]:
from tqdm.notebook import tqdm

In [ ]:
cd ../global-wheat-detection

In [ ]:
train1 = pd.read_csv('train.csv')

In [ ]:
def make_polygon(coords):
    xm, ym, w, h = coords
    xm, ym, w, h = xm / SC_FACTOR, ym / SC_FACTOR, w / SC_FACTOR, h / SC_FACTOR   # scale values if image was downsized
    return [(xm, ym), (xm, ym + h), (xm + w, ym + h), (xm + w, ym)]

masks = dict() # dictionnary containing all masks

for img_id, gp in tqdm(train1.groupby("image_id")):
    gp['polygons'] = gp['bbox'].apply(eval).apply(lambda x: make_polygon(x))

    img = Image.new('L', (IMG_WIDTH, IMG_HEIGHT), 0)
    for pol in gp['polygons'].values:
        ImageDraw.Draw(img).polygon(pol, outline=1, fill=1)

    mask = np.array(img, dtype=np.uint8)
    masks[img_id] = mask

In [ ]:
im = Image.fromarray(masks[list(masks.keys())[7]])
plt.imshow(im)

In [ ]:
def show_images(images, num=2):
    
    images_to_show = np.random.choice(images, num)

    for image_id in images_to_show:

        image_path = os.path.join(train_path, image_id + ".jpg")
        image = Image.open(image_path)
  
        # get all bboxes for given image in [xmin, ymin, width, height]
        bboxes = [literal_eval(box) for box in train1[train1['image_id'] == image_id]['bbox']]

        # visualize them
        draw = ImageDraw.Draw(image)
        for bbox in bboxes:    
            draw.rectangle([bbox[0], bbox[1], bbox[0] + bbox[2], bbox[1] + bbox[3]], width=3)

        plt.figure(figsize = (15,15))
        plt.imshow(image)
        plt.show()


unique_images = train1['image_id'].unique()
show_images(unique_images)

In [ ]:
THRESH = 0.3
masked_preds = preds > THRESH

n_rows = 3

f, ax = plt.subplots(n_rows, 3, figsize=(14, 10))

for j, idx in enumerate([4,5,6]):
    for k, kind in enumerate(['original', 'pred', 'masked_pred']):
        if kind == 'original':
            img = test[idx]
        elif kind == 'pred':
            img = preds[idx]
        elif kind == 'masked_pred':
            masked_pred = preds[idx] > .75
            img = masked_pred
        ax[j, k].imshow(img)

plt.tight_layout()

In [ ]:
def get_params_from_bbox(coords, scaling_factor=1):
    xmin, ymin = coords[1] * scaling_factor, coords[0] * scaling_factor
    w = (coords[3] - coords[1]) * scaling_factor
    h = (coords[2] - coords[0]) * scaling_factor
    
    return xmin, ymin, w, h

In [ ]:
bboxes = list()

for j in range(masked_preds.shape[0]):
    label_j = label(masked_preds) 
    props = regionprops(label_j)
    bboxes.append(props)

In [ ]:
sample_sub = pd.read_csv('sample_submission.csv')

In [ ]:
output = dict()
for i in range(masked_preds.shape[0]):
    bboxes_processed = [get_params_from_bbox(bb.bbox, scaling_factor=SC_FACTOR) for bb in bboxes[i]]
    formated_boxes = ['1.0 ' + ' '.join(map(str, bb_m)) for bb_m in bboxes_processed]
    #if formated_boxes:
    #    formated_boxes = formated_boxes[0] 
    
    output[sample_sub["image_id"][i]] = " ".join(formated_boxes)
    #output[sample_sub["image_id"][i]] = formated_boxes

In [ ]:
sample_sub["PredictionString"] = output.values()


In [ ]:
sample_sub


In [ ]:
cd ..

In [ ]:
cd ../working

In [ ]:
sample_sub.to_csv('submission.csv', index=False)
